In [4]:
!pip install bs4

  Stored in directory: /home/dsxuser/.cache/pip/wheels/a0/b0/b2/4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
Successfully built bs4


### Import libraries

In [6]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

### Obtain data from Wikipedia page

In [8]:
wikiurl = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(wikiurl).text
souptbl = BeautifulSoup(source,'xml')

In [50]:
column_names = ['Postalcode', 'Borough', 'Neighbourhood']
df=pd.DataFrame(columns = column_names)

In [51]:
for tr_cell in souptbl.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data

In [53]:
df['Neighbourhood']=df['Neighbourhood'].apply(lambda x: x.replace('/', ','))
df.head()

,Postalcode,Borough,Neighbourhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park , Harbourfront"


### Data cleaning and formatting

In [56]:
df=df[df['Borough']!='Not assigned']
df=df[df['Borough']!=0]
df.reset_index(drop = True, inplace = True)
i = 0
for i in range(0,df.shape[0]):
    if df.iloc[i][2] == 'Not assigned':
        df.iloc[i][2] = df.iloc[i][1]
        i = i+1
                                 
df = df.groupby(['Postalcode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()

In [73]:
df.head()

,Postalcode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern , Rouge"
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek"
2,M1E,Scarborough,"Guildwood , Morningside , West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [74]:
#find and delete indexes which Borough has value "Not assigned"
df = df.dropna()
empty = 'Not assigned'
df = df[(df.Postalcode != empty ) & (df.Borough != empty) & (df.Neighbourhood != empty)]

In [76]:
def neighbourhood_list(grouped):    
    return ', '.join(sorted(grouped['Neighbourhood'].tolist()))
                    
grp = df.groupby(['Postalcode', 'Borough'])
df2 = grp.apply(neighbourhood_list).reset_index(name='Neighbourhood')

In [77]:
print(df2.shape)
df2.head()

(103, 3)


,Postalcode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern , Rouge"
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek"
2,M1E,Scarborough,"Guildwood , Morningside , West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
